In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import numpy as np
from io import BytesIO
from typing import *
import framework_functions as ff

In [2]:
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\imagenette\imagenette2\output\ds.parquet")

In [4]:
print(ff.draw_tree_schema(table.take([0]).to_pydict()))

├── image_feature
│   ├── [0]
│   │   ├── [0]
│   │   │   ├── boundingbox_feature
│   │   │   │   ├── [0]
│   │   │   │   │   ├── bbox
│   │   │   │   │   │   ├── [0]
│   │   │   │   │   │   ├── [1]
│   │   │   │   │   │   ├── [2]
│   │   │   │   │   │   ├── [3]
│   │   │   │   │   │   ├── [4]
│   │   │   │   │   ├── image_1_feature
│   │   │   │   │   │   ├── [0]
│   │   │   │   │   │   │   ├── image
│   │   │   │   │   │   │   └── shape
│   │   │   │   │   │   │       ├── [0]
│   │   │   │   │   │   │       ├── [1]
│   │   │   │   │   │   │       └── [2]
│   │   │   │   │   ├── label_feature
│   │   │   │   ├── [1]
│   │   │   │   │   ├── bbox
│   │   │   │   │   │   ├── [0]
│   │   │   │   │   │   ├── [1]
│   │   │   │   │   │   ├── [2]
│   │   │   │   │   │   ├── [3]
│   │   │   │   │   │   ├── [4]
│   │   │   │   │   ├── image_1_feature
│   │   │   │   │   ├── label_feature
│   │   │   │   ├── [2]
│   │   │   │   │   ├── bbox
│   │   │   │   │   │   ├── [0]
│   │   │   │   │   │  

In [5]:
root_path = "imagenette/imagenette2/output/partitioning"

In [6]:
ff.partition_dataset(table,root_path,max_batch_size=2)

In [7]:
ff.get_sample(root_path,0,all_data_in_memory=False)

{'bbox': [[0.0977, 0.2604, 0.8789, 0.7812, 1.0]]}

In [8]:
feature_to_merge = [[{"bbox":[i,i,i,i,i]}] for i in range(len(table)*3)]

In [9]:
prova = ff.add_struct_to_feature(table,["image_feature"],"boundingbox_feature",feature_to_merge)

In [10]:
len(prova.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox"))

80

In [15]:
### L'OUTPUT DEVE DIVENTARE 80 (GESTIRE IL NONE); GESTIRE ANCHE LA CARDINALITA' NELL'INSERIMENTO. Esempio: voglio aggiungere bounding boxes solo alla prima immagine di ogni sample
### Una volta generalizzato, riprovare se continua a funzionare con il caso semplice di imagenette (ricarica l'altra tabella)

In [11]:
prova = ff.add_feature(table,["image_feature","boundingbox_feature"],"class_feature",[[{"label":100 + i},{"label":80}] for i in range(len(table)*5)])

In [12]:
len(prova.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("class_feature"))

50

In [13]:
func_doc = {}
func_doc["summary"] = "calculate area of bboxes based on various formats"
func_doc["description"] = "calculate bbox area"
func_name = "area"
in_types = {"array": pa.list_(pa.float64())}
out_type = pa.float64()
pc.register_scalar_function(ff.area, func_name, func_doc, in_types, out_type)

prova = ff.unary_operation_on_feature(table,["image_feature","boundingbox_feature"],"bbox","area","area")

In [14]:
prova.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("area")

[
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  ...
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999
]